In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

In [3]:
# --- Configurações ---
URL = "https://rotasbrasil.com.br/"
rotas_por_origem = {
    "Mato Grosso": [
        'SANTANA', 'PORTO DE MANAUS', 'PORTO SABOÓ', 'Foz do Iguacu(ALF)-PR', 'Foz do Iguacu(IRF)-PR',
        'GUAJARA', 'IRF SAO LUIS', 'PORTO DE VITORIA',
        'PORTO DE PARANAGUA', 'PORTO DE SAO FRANCISCO DO SUL',
        'PORTO DE RIO GRANDE', 'SANTAREM', 'Assis Brasil', 'TABATINGA',
        'ARACAJU', 'PACARAIMA', 'JAGUARAO', 'ITAJAI',
        'AEROPORTO INTERNACIONAL DE VIRACOPOS', 'PORTO VELHO', 'BONFIM',
        'AEROPORTO INTERNACIONAL DE SAO PAULO/GUARULHOS', 'BARCARENA',
        'CORUMBA', 'BRASILEIA', 'PONTA PORA', 'PORTO DE SAO LUIS',
        'IMBITUBA', 'FOZ DO IGUACU', 'CHUI', 'PORTO DE SALVADOR',
        'CACERES', 'SAO PAULO'
    ],
    "Goias": [
        'PORTO SABOÓ', 'AEROPORTO INTERNACIONAL DE VIRACOPOS',
        'PORTO DE PARANAGUA', 'Foz do Iguacu(ALF)-PR', 'PORTO DE VITORIA',
        'PORTO DE SAO FRANCISCO DO SUL', 'PORTO DE RIO GRANDE', 'Foz do Iguacu(IRF)-PR',
        'ITAJAI', 'IRF SAO LUIS',
        'AEROPORTO INTERNACIONAL DE SAO PAULO/GUARULHOS', 'JAGUARAO',
        'PACARAIMA', 'Assis Brasil', 'IRF CAMPOS DOS GOYTACAZES',
        'PORTO DE MANAUS', 'IRF ILHEUS', 'GUAJARA', 'FOZ DO IGUACU',
        'PORTO DE SAO LUIS', 'PORTO DE SALVADOR', 'PONTA PORA', 'IMBITUBA',
        'CHUI', 'SANTAREM', 'PORTO DO RIO DE JANEIRO', 'PORTO DE BELEM'
    ],
    "Mato Grosso do Sul": [
        'PORTO DE PARANAGUA', 'Foz do Iguacu(ALF)-PR', 'PORTO SABOÓ',
        'PORTO DE SAO FRANCISCO DO SUL', 'PORTO DE RIO GRANDE', 'JAGUARAO',
        'ARF', 'IRF Mato Grosso do Sul', 'Assis Brasil', 'PACARAIMA', 'PORTO DE VITORIA',
        'AEROPORTO INTERNACIONAL DE SAO PAULO/GUARULHOS', 'ITAJAI',
        'AEROPORTO INTERNACIONAL DE VIRACOPOS', 'PORTO MURTINHO',
        'PONTA PORA', 'GUAJARA', 'SANTAREM', 'IMBITUBA', 'CORUMBA'
    ],
    "Alto Paraná": [
        'PORTO DE PARANAGUA', 'Foz do Iguacu(ALF)-PR', 'IRF SAO LUIS', 'PORTO SABOÓ',
        'PORTO DE SAO FRANCISCO DO SUL', 'Foz do Iguacu(IRF)-PR',
        'AEROPORTO INTERNACIONAL DE VIRACOPOS',
        'AEROPORTO INTERNACIONAL DE SAO PAULO/GUARULHOS', 'SAO SEBASTIAO',
        'JAGUARAO', 'PORTO DE RIO GRANDE', 'Assis Brasil', 'PACARAIMA',
        'BAGE', 'ITAJAI', 'IRF SANTANA DO LIVRAMENTO', 'SANTAREM',
        'PORTO DE VITORIA', 'ILHEUS', 'FOZ DO IGUACU', 'URUGUAIANA',
        'PONTA PORA', 'IMBITUBA', 'SANTA HELENA', 'GUAIRA',
        'PORTO DE MANAUS', 'PORTO DO RIO DE JANEIRO', 'GUAJARA',
        'SANTANA DO LIVRAMENTO',
        'AEROPORTO INTERNACIONAL DO RIO DE JANEIRO', 'MUNDO NOVO'
    ],
    "Rio Grande do Sul": [
        'Foz do Iguacu(ALF)-PR', 'Foz do Iguacu(IRF)-PR', 'PORTO DE PARANAGUA',
        'PORTO DE SAO FRANCISCO DO SUL', 'PORTO DE RIO GRANDE',
        'AEROPORTO INTERNACIONAL DE SAO PAULO/GUARULHOS',
        'AEROPORTO INTERNACIONAL DE VIRACOPOS', 'BAGE',
        'AEROPORTO INTERNACIONAL DE BRASILIA', 'PORTO SABOÓ', 'ITAJAI',
        'IRF SANTANA DO LIVRAMENTO', 'IRF PORTO XAVIER', 'JAGUARAO',
        'IRF SAO LUIS', 'PORTO XAVIER', 'GUAJARA', 'URUGUAIANA',
        'PORTO DE SAO LUIS', 'SANTANA DO LIVRAMENTO', 'CHUI',
        'FOZ DO IGUACU', 'CORUMBA', 'SAO BORJA', 'IMBITUBA',
        'PORTO DE MANAUS'
    ]
}

dados_coletados = []

'SANTANA', 'PORTO DE MANAUS', 'PORTO SABOÓ', 'ALF', 'IRF',
       'GUAJARA', 'IRF SAO LUIS', 'PORTO DE VITORIA',
       'PORTO DE PARANAGUA', 'PORTO DE SAO FRANCISCO DO SUL',
       'PORTO DE RIO GRANDE', 'SANTAREM', 'Assis Brasil', 'TABATINGA',
       'ARACAJU', 'PACARAIMA', 'JAGUARAO', 'ITAJAI',
       'AEROPORTO INTERNACIONAL DE VIRACOPOS', 'PORTO VELHO', 'BONFIM',
       'AEROPORTO INTERNACIONAL DE SAO PAULO/GUARULHOS', 'BARCARENA',
       'CORUMBA', 'BRASILEIA', 'PONTA PORA', 'PORTO DE SAO LUIS',
       'IMBITUBA', 'FOZ DO IGUACU', 'CHUI', 'PORTO DE SALVADOR',
       'CACERES', 'SAO PAULO'

In [4]:
driver = webdriver.Chrome() 
driver.get(URL)
wait = WebDriverWait(driver, 15) 

# --- 3. SELEÇÃO DO CAMINHÃO (Bloco try/except separado) ---

try:
    botao_caminhao = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "caminhao"))) 
    botao_caminhao.click()
    print("🚛 Selecionado o modo Caminhão.")
    time.sleep(1) 
except Exception as e:
    print(f"⚠️ Não foi possível clicar no botão do caminhão. Erro: {e}")
    time.sleep(1)

# --- 4. LOOP PRINCIPAL PARA MÚLTIPLAS ORIGENS ---

# O bloco TRY/EXCEPT externo captura erros críticos (ex: navegador trava)
try:
    # Loop EXTERNO: Itera sobre cada estado de origem e sua lista de destinos
    for PONTO_DE_PARTIDA, portos_destino_list in rotas_por_origem.items():
        
        print(f"\n=======================================================")
        print(f"🔄 INICIANDO ROTAS PARA ORIGEM: {PONTO_DE_PARTIDA}")
        print(f"=======================================================")
        
        # 1. ENCONTRA E PREENCHE O CAMPO DE ORIGEM (apenas UMA VEZ por estado)
        # Re-esperamos o elemento para garantir que está pronto após o preenchimento/cálculo anterior
        campo_origem = wait.until(EC.presence_of_element_located((By.ID, "txtEnderecoPartida")))
        campo_origem.clear() 
        campo_origem.send_keys(PONTO_DE_PARTIDA)
        time.sleep(1) 
        
        # Loop INTERNO: Itera sobre cada destino da lista do estado atual
        for destino in portos_destino_list:
            print(f"  Buscando rota para: {destino}...")
            
            # --- ETAPAS 2, 3: PREENCHIMENTO DO DESTINO E CLIQUE ---
            
            # 2. Encontra e preenche o campo de DESTINO
            campo_destino = driver.find_element(By.ID, "txtEnderecoChegada")
            campo_destino.clear()
            campo_destino.send_keys(destino)
            time.sleep(1) 
            
            # 3. Clica no botão "BUSCAR ROTA"
            botao_calcular = wait.until(EC.element_to_be_clickable((By.ID, "btnSubmit")))
            driver.execute_script("arguments[0].click();", botao_calcular)

            # ----------------------------------------------------
            # ETAPA 4: COLETA DOS RESULTADOS (TRY/EXCEPT interno para falhas na coleta)
            try:
                # 4.1. Coleta a DISTÂNCIA (classe 'distance')
                distancia_elem = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "distance")))
                distancia = distancia_elem.text.strip()
                
                # 4.2. Coleta o TEMPO (XPATH exato)
                tempo_elem = driver.find_element(
                    By.XPATH, 
                    "//div[@class='col-6 mr-auto text-left h3 color-primary-500']"
                )
                tempo = tempo_elem.text.strip()

                # 4.3. Coleta as RODOVIAS (classe 'titulo')
                rodovias_elem = driver.find_element(By.CLASS_NAME, "titulo")
                texto_rodovias = rodovias_elem.text.replace("via", "").strip()

                # 4.4 Custo total
                custo_total = driver.find_element(By.CLASS_NAME,"nowrap") 
                custo = custo_total.text.strip()
                

                # ADICIONA RESULTADO COM SUCESSO
                dados_coletados.append({
                    'Origem': PONTO_DE_PARTIDA,
                    'Destino': destino,
                    'Distancia': distancia,
                    'Tempo_Estimado': tempo,
                    'Rodovias': texto_rodovias,
                    'Custo': custo
                })
                
                print(f"    ✅ Sucesso! Distância: {distancia}")
                
            except Exception as e:
                # ADICIONA RESULTADO COM FALHA
                print(f"    ❌ Falha na coleta dos dados para {destino}. Erro: {e}")
                dados_coletados.append({
                    'Origem': PONTO_DE_PARTIDA, 
                    'Destino': destino, 
                    'Distancia': 'FALHA', 
                    'Tempo_Estimado': 'FALHA', 
                    'Rodovias': 'FALHA', 
                    'Custo': 'FALHA'
                })

            # 5. Pausa de segurança, DENTRO do loop interno
            time.sleep(3) 

# --- FIM DO LOOP ---

except Exception as e:
    print(f"\n🚨 ERRO GRAVE NO PROCESSO: O loop principal falhou. Erro: {e}")

# --- 5. FECHAMENTO DO DRIVER ---
finally:
    # Fecha o navegador, garantindo a liberação de recursos.
    driver.quit()

# --- 6. SALVAMENTO DOS RESULTADOS ---
# Renomeado o arquivo para refletir que a coleta é de múltiplas origens
if dados_coletados:
    df_resultado = pd.DataFrame(dados_coletados)
    
    # Ordem das colunas
    colunas_ordenadas = ['Origem', 'Destino', 'Distancia', 'Tempo_Estimado', 'Rodovias','Custo']
    df_resultado = df_resultado[colunas_ordenadas] 
    
    # Nome do arquivo mais genérico
    num_origens = len(rotas_por_origem)
    num_total_rotas = len(dados_coletados)
    nome_arquivo = f"rotas_{num_origens}_origens_{num_total_rotas}_rotas.xlsx"
    
    df_resultado.to_excel(nome_arquivo, index=False)
    print("\n" + "="*50)
    print(f"✨ Coleta concluída! Dados salvos em: {nome_arquivo}")
    print("="*50)

🚛 Selecionado o modo Caminhão.

🔄 INICIANDO ROTAS PARA ORIGEM: Mato Grosso
  Buscando rota para: SANTANA...
    ✅ Sucesso! Distância: 1.885,3 km
  Buscando rota para: PORTO DE MANAUS...
    ✅ Sucesso! Distância: 2.400,6 km
  Buscando rota para: PORTO SABOÓ...
    ✅ Sucesso! Distância: 2.077,3 km
  Buscando rota para: Foz do Iguacu(ALF)-PR...
    ✅ Sucesso! Distância: 1.882,0 km
  Buscando rota para: Foz do Iguacu(IRF)-PR...
    ✅ Sucesso! Distância: 1.884,3 km
  Buscando rota para: GUAJARA...
    ✅ Sucesso! Distância: 2.677,6 km
  Buscando rota para: IRF SAO LUIS...
    ✅ Sucesso! Distância: 2.439,0 km
  Buscando rota para: PORTO DE VITORIA...
    ✅ Sucesso! Distância: 2.198,4 km
  Buscando rota para: PORTO DE PARANAGUA...
    ✅ Sucesso! Distância: 2.273,3 km
  Buscando rota para: PORTO DE SAO FRANCISCO DO SUL...
    ✅ Sucesso! Distância: 2.356,3 km
  Buscando rota para: PORTO DE RIO GRANDE...
    ✅ Sucesso! Distância: 2.598,5 km
  Buscando rota para: SANTAREM...
    ✅ Sucesso! Distânc